In [1]:
import sys
import time
sys.path.insert(0,'../') #enable loading rxn package

import jgraph
import imolecule
from rxn.data_structures import MolGraph, RxnGraph
from rxn.rxn_network import scissions, recursive_scissions

We can start by generating all of the scission reactions for the HONHNHOH molecule. Note that the visualization of the reaction network is not very useful without labels, which still needs to be figured out and will require modifying or wrapping the jgraph library, or finding another visualization library. The conversion to `rxn_list` is also ugly, and in general the input/output functionality of RxnGraph is still under development.

In [2]:
ONNO = MolGraph().generate('ONNO','smi') #generate ONNO
rxns = scissions(ONNO)

rxns_list = []
for rxn in rxns:
    products = [rxn.node[p]['graph'] for p in rxn.nodes() if rxn.node[p]['type']=='molecule' and rxn.node[p]['molecule_type']=='product']
    reactants = [rxn.node[p]['graph'] for p in rxn.nodes() if rxn.node[p]['type']=='molecule' and rxn.node[p]['molecule_type']=='reactant']
    rxns_list.append([reactants, products])

all_rxns = RxnGraph()
all_rxns.from_rxn_list(rxns_list)

print(all_rxns)
jgraph.draw(all_rxns.to_jgraph())

HOHNHNOH->H+HONHNOH
HOHNHNOH->HNHO+HOHN
HOHNHNOH->H+HOHNHNO
HOHNHNOH->HHNNOH+HO


Next we can look at the entire reaction network for conversion of N2, H2O, and O2 to NH3, NO, and NH2OH with the largest intermediate species being HONHNHOH, HONHNH2, or NH2NH2.

In [3]:
ONNO = MolGraph().generate('ONNO','smi') #generate ONNO
ONN = MolGraph().generate('ONN','smi') #generate ONN
N2H4 = MolGraph().generate('NN','smi') #generate N2H4
NH3 = MolGraph().generate('[NH3]','smi') #generate NH3
H2O = MolGraph().generate('[OH2]','smi') #generate H2O
O2 = MolGraph().generate('O=O','smi') #generate O2
N2 = MolGraph().generate('N#N','smi') #generate N2
NH2OH = MolGraph().generate('NO','smi') #generate NH2OH

species = []
rxns = recursive_scissions([ONNO, ONN, N2H4, NH3, H2O, N2, O2, NH2OH],finished=species)

rxns_list = []
for rxn in rxns:
    products = [rxn.node[p]['graph'] for p in rxn.nodes() if rxn.node[p]['type']=='molecule' and rxn.node[p]['molecule_type']=='product']
    reactants = [rxn.node[p]['graph'] for p in rxn.nodes() if rxn.node[p]['type']=='molecule' and rxn.node[p]['molecule_type']=='reactant']
    rxns_list.append([reactants, products])

all_rxns = RxnGraph()
all_rxns.from_rxn_list(rxns_list)

Recursing on:
['HOHNHNO', 'HOHNHN', 'OH', 'HNOH', 'HHOHNNO', 'HONHNH', 'HNH', 'HHNNH', 'HONHNH', 'HOHN']
Recursing on:
['HNO', 'HOHNNO', 'HONHNO', 'HOHNN', 'OHNHNO', 'HN', 'HNNHO', 'HNNHO', 'HHNN', 'NOH', 'HHONNO', 'NHNH', 'ONHNH']
Recursing on:
['NO', 'ONNHO', 'HONNO', 'HONN', 'OHNN', 'NNH', 'NNHO']
Recursing on:
['ONN', 'ONNO']


Note that this takes a bit of time sense it needs to recurse through several layers. Before we visualize the network we can first look at all the species involved. Note that we need to "sleep" for 1 second in between to allow the visualization library time to render each graph.

In [7]:
for i, sp in enumerate(species):
    print('species[{}] = {}'.format(i,sp))
    json = sp.to_dict()
    time.sleep(1)
    imolecule.draw(json,format='json')

species[0] = HHOHNHNO


species[1] = HHONHNH


species[2] = HHHNNH


species[3] = HHNH


species[4] = HOH


...

species[35] = OHNN


species[36] = NNH


species[37] = NNHO


species[38] = ONN


species[39] = ONNO


These species compries the full reaction network. Let's see how many species and reactions are involved.

In [5]:
print('Total # of species: {}'.format(len(species)))
print('Total # of reactions: {}'.format(len(all_rxns)))

Total # of species: 40
Total # of reactions: 179


This means that to get a comprehensive picture of the reaction network we need the energies for 40 species. If we wanted to build a full kinetic model we would also need 177 reaction barriers, which is borderline intractible. Finally we can visualize the full network, though it won't mean much without labels.

In [6]:
print(all_rxns)
jgraph.draw(all_rxns.to_jgraph())

HOHNHNO->HOHNHN+O
OHNN->N+HNO
HOHNHNO->H+HONHNO
NNHO->HO+NN
NOH->H+NO
OHNHNO->HNO
ONNHO->H+ONNO
HOHNHNO->H+HOHNNO
HONHNO->H+ONNHO
HHHNNH->HNH
HOHNHN->H+HHNNO
HHNOH->HNOH+H
HOHN->H+OHN
NOH->N+OH
OHNHNO->H+HONNO
HOHN->O+HHN
HONHNO->OHNN+HO
HOH->H+HO
HONN->H+ONN
HONHNH->H+HNONH
HONHNO->O+HNNHO
HHOHNHNO->HOHNHN+OH
HHNNH->HNNH+H
HONHNH->ONHNH+H
HHOHNNO->NOH+HNHO
HNOH->NH+OH
HHONHNH->HNH+HONH
HOHNHNO->OHNHNO+H
HHNN->H+HNN
HNOH->NOH+H
HHHNNH->HHNNH+H
NO->N+O
HNOH->HNO+H
HHOHNNO->H+HOHNNO
HNNHO->H+NNHO(1)
HHNNH->HHNN+H
HHONHNH->H+HONHNH
HOHNHNO->HOHNN+HO
HONNO->NO+HNO
HNNHO->H+HNNO(1)
ONHNH->ON+HNH
HOHNN->H+ONNH
NN->N
HHNH->HNH+H
ONHNH->HHNN+O
HOHNN->NH+HNO
HHOHNNO->H+HOHNNO(1)
HONHNH->ONH+HNH
OHNN->ONN+H
HHNOH->HHN+OH
HONHNH->HNH+NOH
HHNN->NH
HHOHNHNO->HNOH+HNHO
HOHNHN->H+HNNHO
HOHNNO->O+HNNHO
HONHNO->H+OHNNO
HONNO->H+ONNO
HNH->HN+H
HONNO->HNNO+O
ONNHO->HO+ONN
HNNHO->NNH+HO
ONNHO->O+NNHO
HOHNNO->H+HONNO
NNHO->N+NHO
HHONNO->NHO+NOH
HONN->NH+NO
HOHNHN->HN+HNHO
HONHNH->NHNH+OH
HONHNH->HNNOH+H
HH

There isn't much we can learn from this, but it's a safe bet that the central point is hydrogen.